In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Read Data in and do head

# Show distribution of the categories

# Exploratory Data Analysis (plot average sleep for each category with seaborn barplot)

# Correlation heatmap sns.heatmap(df.corr())

# Narrow down what features we want through correlation and other exploratory data analysis, comparing features across stress level categories.

# train test split to get X_train, X_test, y_train, y_test. Check distribution after split.

# train test each model (if we have time try MLP)

# cross validation of models

# evaluation metrics and run time evaluation

# baseline of mode and random off of entire datset

# create a small dataframe with model name that shows accuracy, precision, recall, runtime

# learning curve for each model? If curve needs data, maybe add synthetic data?

# Hyperparameter Tuning

# Figure out best model and do confusion matrix

# Feature importance to see what sleep features matter